<h1>Question 1<h1>

<h2>Loading Necessary Libraries<h2>

In [2]:
#Basics
import numpy as np
import pandas as pd 

#JSON
import json
import requests
from pandas.io.json import json_normalize 

#Geopy
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

# Plotting
import matplotlib.cm as mpcm
import matplotlib.colors as mpcol

#SKLearn for KMN
from sklearn.cluster import KMeans

# BeautifulSoup for Webscrape
from bs4 import BeautifulSoup

#XML Reader
import xml

print('OK done')

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

<h2> Get data from Wikipedia<h2>

In [4]:
#Scrape
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(link,'lxml')
#locate Table
table_post = soup.find('table')
fields = table_post.find_all('td')

#Create Table
postcode = []
borough = []
neighborhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighborhood.append(fields[i+2].text.strip())
        
df_poscod = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_poscod.columns = ['Postcode', 'Borough', 'Neighborhood']
df_poscod.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h2> Data Wrangling<h2>

In [8]:
#Delete Not Assigned
df_poscod['Borough'].replace('Not assigned', np.nan, inplace=True)
df_poscod.dropna(subset=['Borough'], inplace=True)

#Group by Postcode & Borough
df_poscod2 = df_poscod.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_poscod2.columns = ['Postcode', 'Borough', 'Neighborhood']

#Replace Not Assigned Neighborhood to Queen's Park
df_poscod2['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

df_poscod2

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [11]:
#Check Shape
df_poscod2.shape

(103, 3)

<h1>Question 2<h1>

<h2>Create Toronto DF<h2>

In [17]:
#Load Geoloc, Merge Postcode
df_g = pd.read_csv('http://cocl.us/Geospatial_data')
df_g.columns = ['Postcode', 'Latitude', 'Longitude']
df_match = pd.merge(df_poscod2, df_g, on=['Postcode'], how='inner')
df_match2 = df_match[['Borough', 'Neighborhood', 'Postcode', 'Latitude', 'Longitude']].copy()
df_match2.head()

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [19]:
#Filter out Non-Toronto
df_toro = df_match2[df_match2['Borough'].str.contains('Toronto')]
dat_toro = df_toro.reset_index(drop=True)
dat_toro

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
5,Central Toronto,Davisville North,M4P,43.712751,-79.390197
6,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
7,Central Toronto,Davisville,M4S,43.704324,-79.388790
8,Central Toronto,"Moore Park, Summerhill East",M4T,43.689574,-79.383160
9,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V,43.686412,-79.400049


<h1> Question 3<h1>

<h2>Explore Toronto<h2>

<h4>In this section we will get list and count of Toronto venues through Foursquare API (up to 100 in 500 M radius)<h4>

In [21]:
#Foursq Credentials
CLIENT_ID = 'GSSY4GWNQGVOSNI0KJIZR1RDEVSBXRPGAW3BJCXYEGJLVAV2' # your Foursquare ID
CLIENT_SECRET = 'U3AKXJWQVVSZCDQ2L0UQCLSBERTVNHAJTFBTXOVPYT2NATZY' # your Foursquare Secret
VERSION = '20190505' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GSSY4GWNQGVOSNI0KJIZR1RDEVSBXRPGAW3BJCXYEGJLVAV2
CLIENT_SECRET:U3AKXJWQVVSZCDQ2L0UQCLSBERTVNHAJTFBTXOVPYT2NATZY


In [22]:
#Get Lat Long of Toronto

address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GSSY4GWNQGVOSNI0KJIZR1RDEVSBXRPGAW3BJCXYEGJLVAV2&client_secret=U3AKXJWQVVSZCDQ2L0UQCLSBERTVNHAJTFBTXOVPYT2NATZY&v=20190505&ll=43.653963,-79.387207&radius=500&limit=100'

In [25]:
#Get JSON
results = requests.get(url).json()

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
#JSON to Venues
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Cafe Plenty,Café,43.654571,-79.389450
4,Tsujiri,Tea Room,43.655374,-79.385354


<h4>How many venues in 500 radius ? (Limit 100)<h4>

In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

81 venues were returned by Foursquare.


<h2>Explore Toronto Deeper<h2>

In [32]:
#Define getNerarbyVenues function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
#Get Nearby Venues of Toronto

toronto_venues = getNearbyVenues(names=dat_toro['Neighborhood'],
                                   latitudes=dat_toro['Latitude'],
                                   longitudes=dat_toro['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [34]:
#Check Dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1690, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678730,-79.297478,Grocery Store
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


<h4>Count of venues in each neighborhood<h4>

In [35]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


<h4>We can understand how many unique categories<h4>

In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[162]] + list(toronto_onehot.columns[:162])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
#Check the Shape
toronto_onehot.shape

(1690, 163)

In [51]:
#Group by Neighborhood to get frequency
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.017857,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011628,...,0.000000,0.023256,0.011628,0.011628,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.040000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.011364,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


<h4>Print each Neighborhood and 5 top venues<h4>

In [52]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2                  Bar  0.04
3  American Restaurant  0.04
4         Burger Joint  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3         Cheese Shop  0.04
4  Italian Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                    Café  0.11
1          Breakfast Spot  0.11
2             Coffee Shop  0.11
3                     Bar  0.05
4  Furniture / Home Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1             Brewery  0.06
2          Comic Shop  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
           

<h4>Put into the dataframe<h4>

In [53]:
#Define Most Common Venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Bar,Hotel,Cosmetics Shop,Bakery,Burger Joint,Japanese Restaurant,Concert Hall
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Italian Restaurant,Farmers Market,Beer Bar,Café,Gourmet Shop,Irish Pub
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Furniture / Home Store,Convenience Store,Gym,Grocery Store,Caribbean Restaurant,Italian Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Brewery,Garden Center,Garden,Butcher,Comic Shop,Burrito Place,Fast Food Restaurant,Farmers Market
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Colombian Restaurant


<h2>Clustering Neighborhood<h2>

Run k-means to cluster the neighborhood into 6 clusters.

In [57]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 0, 0, 4, 4, 4, 4, 4], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = dat_toro

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Postcode,Latitude,Longitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,M4E,43.676357,-79.293031,5,5,Health Food Store,Grocery Store,Music Venue,Chocolate Shop,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188,4,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Fruit & Vegetable Store,Juice Bar,Bakery,Liquor Store
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572,0,0,Brewery,Fast Food Restaurant,Gym,Ice Cream Shop,Italian Restaurant,Burrito Place,Liquor Store,Food & Drink Shop,Fish & Chips Shop,Burger Joint
3,East Toronto,Studio District,M4M,43.659526,-79.340923,4,4,Café,Coffee Shop,Bakery,American Restaurant,Gastropub,Italian Restaurant,Latin American Restaurant,Gym / Fitness Center,Bank,Bar
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,3,3,Bus Line,Burrito Place,Colombian Restaurant,College Rec Center,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym


<h2>Create visualization<h2>

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = mpcm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [mpcol.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Examining clusters<h2>

<h4>Cluster 1<h4>

In [64]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"The Beaches West, India Bazaar",0,0,Brewery,Fast Food Restaurant,Gym,Ice Cream Shop,Italian Restaurant,Burrito Place,Liquor Store,Food & Drink Shop,Fish & Chips Shop,Burger Joint
8,"Moore Park, Summerhill East",0,0,Music Venue,Burrito Place,Colombian Restaurant,College Rec Center,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym
10,Rosedale,0,0,Music Venue,Burrito Place,Colombian Restaurant,College Rec Center,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym
23,"Forest Hill North, Forest Hill West",0,0,Jewelry Store,Music Venue,Colombian Restaurant,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym,Church
27,"CN Tower, Bathurst Quay, Island airport, Harbo...",0,0,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Colombian Restaurant
31,"Dovercourt Village, Dufferin",0,0,Bakery,Music Venue,Middle Eastern Restaurant,Gym / Fitness Center,Brewery,Bar,Bank,Liquor Store,Café,Discount Store
35,"Parkdale, Roncesvalles",0,0,Gift Shop,Breakfast Spot,Eastern European Restaurant,Dessert Shop,Bank,Coffee Shop,Italian Restaurant,Cuban Restaurant,Bookstore,Dog Run
37,Business Reply Mail Processing Centre 969 Eastern,0,0,Light Rail Station,Auto Workshop,Brewery,Garden Center,Garden,Butcher,Comic Shop,Burrito Place,Fast Food Restaurant,Farmers Market


<h4>Cluster 2<h4>

In [66]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,1,1,Garden,Burrito Place,Colombian Restaurant,College Rec Center,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym


<h4>Cluster 3<h4>

In [67]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Davisville North,2,2,Hotel,Breakfast Spot,Clothing Store,Gym,Food & Drink Shop,Music Venue,Chocolate Shop,College Arts Building,Coffee Shop,Cocktail Bar


<h4>Cluster 4<h4>

In [68]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,3,3,Bus Line,Burrito Place,Colombian Restaurant,College Rec Center,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym


<h4>Cluster 5<h4>

In [69]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"The Danforth West, Riverdale",4,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Fruit & Vegetable Store,Juice Bar,Bakery,Liquor Store
3,Studio District,4,4,Café,Coffee Shop,Bakery,American Restaurant,Gastropub,Italian Restaurant,Latin American Restaurant,Gym / Fitness Center,Bank,Bar
6,North Toronto West,4,4,Coffee Shop,Clothing Store,Bagel Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Cosmetics Shop,Chinese Restaurant
7,Davisville,4,4,Dessert Shop,Café,Italian Restaurant,Coffee Shop,American Restaurant,Fried Chicken Joint,Indian Restaurant,Bowling Alley,Greek Restaurant,Gourmet Shop
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",4,4,Coffee Shop,Light Rail Station,American Restaurant,Fried Chicken Joint,Liquor Store,Bagel Shop,Caribbean Restaurant,Cajun / Creole Restaurant,Colombian Restaurant,Butcher
11,"Cabbagetown, St. James Town",4,4,Coffee Shop,Café,Italian Restaurant,Market,Bakery,Breakfast Spot,Jewelry Store,Japanese Restaurant,Indian Restaurant,Beer Store
12,Church and Wellesley,4,4,Coffee Shop,Japanese Restaurant,Gay Bar,Gastropub,Café,Mediterranean Restaurant,Men's Store,Burger Joint,Gym,Fast Food Restaurant
13,"Harbourfront, Regent Park",4,4,Coffee Shop,Bakery,Breakfast Spot,Mexican Restaurant,Café,Electronics Store,Cosmetics Shop,Health Food Store,Historic Site,Brewery
14,"Ryerson, Garden District",4,4,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Bakery,Bubble Tea Shop
15,St. James Town,4,4,Coffee Shop,Café,Hotel,Cosmetics Shop,Clothing Store,Breakfast Spot,Bakery,Gastropub,Japanese Restaurant,Cocktail Bar


<h4>Cluster 6<h4>

In [70]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,5,5,Health Food Store,Grocery Store,Music Venue,Chocolate Shop,College Gym,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym
